In [10]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
colnames=['Date', 'IND1', 'IND2', 'IND3', 'IND4', 'IND5', 'IND6'] 
households = pd.read_csv("https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=1&p_sort=2&p_des=50&p_sestuid=27368&p_uka=1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11&p_strid=AABBAF&p_od=199301&p_do=203012&p_lang=EN&p_format=2&p_decsep=.",sep='|')
nonperforming = pd.read_csv("https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=1&p_sort=2&p_des=50&p_sestuid=52952&p_uka=1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11&p_strid=AABBAF&p_od=200201&p_do=203012&p_lang=EN&p_format=2&p_decsep=.",sep='|')
new_business = pd.read_csv("https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=1&p_sort=2&p_des=50&p_sestuid=58858&p_uka=1%2C2%2C3%2C4%2C5%2C6&p_strid=AAABAB&p_od=200401&p_do=203012&p_lang=EN&p_format=2&p_decsep=.", sep='|', names=colnames,skiprows=1)

In [11]:
def gen_chart(field, title, chart_type):
    if chart_type == 'TotalVolume':
        df=households[['Date',field]].dropna()
    else:
        df=new_business[['Date',field[0:4]]].dropna()
    df['Date'] =  pd.to_datetime(df['Date'],format='%d.%m.%Y')
    df.set_index('Date',inplace=True)
    df.sort_index(inplace=True)
    if chart_type == 'TotalVolume':
        df.rename(columns={field: 'Total Loans'}, inplace=True)
    else:
        df.rename(columns={field[0:4]: 'Total Loans'}, inplace=True)
    df['Total Loans YoY Change']=df['Total Loans'].pct_change(periods=12)
    df=df.tail(13)

    if chart_type == 'TotalVolume':
        dfn=nonperforming[['Date',field +' - non-performing']].dropna()
        dfn['Date'] =  pd.to_datetime(dfn['Date'],format='%d.%m.%Y')
        dfn.set_index('Date',inplace=True)
        dfn.sort_index(inplace=True)
        dfn.rename(columns={field +' - non-performing': 'Total Loans Nonperforming'}, inplace=True)
        dfn=dfn.tail(13)

    if chart_type == 'TotalVolume':
        df_chart = pd.concat([df, dfn], axis=1, join='inner')
    else:
        df_chart = df.copy()
        
    if chart_type == 'TotalVolume':
        df_chart['Total Loans Nonperforming Rate']=df_chart['Total Loans Nonperforming']/df_chart['Total Loans']    
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
       go.Scatter(
                    x=df_chart.index,
                    y=df_chart['Total Loans YoY Change'],
                    name='YoY Change (%)',
                    mode='lines+markers+text',
                    text=round(df_chart['Total Loans YoY Change'],4),
                    texttemplate = "%{text:.2%}",
                    textposition="top center",
                    textfont = {'size': 10},
                    hoverinfo='none'
                    ),
        secondary_y=True)

    if chart_type == 'TotalVolume':
        fig.add_trace(
           go.Scatter(
                        x=df_chart.index,
                        y=df_chart['Total Loans Nonperforming Rate'],
                        name='Non-performing Rate (%)',
                        mode='lines+markers+text',
                        text=round(df_chart['Total Loans Nonperforming Rate'],4),
                        texttemplate = "%{text:.2%}",
                        textposition="bottom center",
                        textfont = {'size': 10},
                        hoverinfo='none'
                        ),
            secondary_y=True)

    fig.add_trace(
        go.Bar(
                x=df_chart.index,
                y=df_chart['Total Loans']/1000,
                name='Loans (CZK billions)',
                marker_color='rgb(158,202,225)', 
                text=round(df_chart['Total Loans']/1000,0),
                texttemplate = "%{text:,.0f}",
                textposition = "auto",
                textfont = {'size': 10},
                hoverinfo='none'
                ),
        secondary_y=False)

    Chart_Title_Font_Size=16
    yaxis_font_size=12
    xaxis_font_size=12
    
    if chart_type == 'TotalVolume':
        y_range=[0, 0.15]
    else:
        y_range=[-0.4, 0.4]

    fig.update_layout(
            autosize=False,
            width=1000,
            height=400,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            showlegend=True,
            title=title, 
            font=dict(size=Chart_Title_Font_Size),
            yaxis=dict(title="Loans (CZK)", tickformat=",.0f",titlefont=dict(size=yaxis_font_size), 
                              tickfont=dict(size=yaxis_font_size), visible=False
                             ),
            yaxis2=dict(title='YoY Change(%)/Non-performing Rate(%)',tickformat=".2%", titlefont=dict(size=yaxis_font_size), 
                              tickfont=dict(size=yaxis_font_size), visible=False, range=y_range),
            xaxis=dict(title='Calendar Month', titlefont=dict(size=xaxis_font_size), 
                       tickfont=dict(size=xaxis_font_size),
                       tickformat="%b%y",
                       tickmode='array',
                       ticktext=df_chart.index
                      ),
            legend=go.layout.Legend(
                                        font=dict(
                                            family="sans-serif",
                                            size=12,
                                            color="black"
                                        )
                                    )
                  )
    fig.show()

# Czech Banking Statistics 
CZK-denominated Loans (in billions)<br /> 
Source: Czech National Bank, ARAD data series system

In [12]:
gen_chart('IND1 Residents-loans to households (incl. NPISHs) total', 'Loans to Households Total', 'TotalVolume')

In [13]:
gen_chart('IND2 consumer credit to households incl.debit balances on current account total', 'Consumer Credit Loans', 'TotalVolume')

In [14]:
gen_chart('IND6 lending to households for house purchase total', 'House Purchase Loans', 'TotalVolume')

In [15]:
gen_chart('IND7 of which: mortgage loans to households for house purchase total', 'House Purchase Loans - Mortgage', 'TotalVolume')

In [16]:
gen_chart('IND1 Residents-loans to households (incl. NPISHs) total', 'Loans to Households Total: New Business', 'NewVolume')

In [18]:
gen_chart('IND2 consumer credit to households incl.debit balances on current account total', 'Consumer Credit Loans: New Business', 'NewVolume')

In [17]:
gen_chart('IND6 lending to households for house purchase total', 'House Purchase Loans: New Business', 'NewVolume')